In [1]:
from tensorflow.keras import utils
import numpy as np

path = utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower() # 말물칭를 내려받아 소문자로 바꿔준다.
print('말뭉치 크기:', len(text))

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [15]:
maxlen = 60 # 60개의 글자로 된 시퀀스 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링한다.

sentences = [] # 추출한 시퀀스를 담을 리스트

next_chars = [] # 타깃(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))  # 200278

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자:', len(chars)) # 58
char_indices = dict((char, char.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

print('벡터화...')
# 글자를 원핫 인코딩하여 0과 1의 이진 배열로 바꿔준다.
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1 
    

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


In [8]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [11]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [13]:
import numpy as np

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60): # 60 에포크 동안 모델 훈련
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1) # 데이터에서 한 번만 반복해서 모델 학습
    
    seed_text = text[start_index: start_index + maxlen] # 무작위로 시드 텍스트를 선택한다.
    print('--- 시드 텍스트: "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: # 여러 가지 샘플링 온도를 시도
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400): # 시드 텍스트에서 시작해서 400개의 글자를 생성한다.
            # 원 핫 인코딩으로 바꿈
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text): 
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 43s 28ms/step - loss: 0.0000e+00
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through fo
















































































































































































































































































































































































































------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through fo



































































































































































































































































































KeyboardInterrupt: 